In [1]:
import numpy as np 
import matplotlib.pyplot as plt
#for part e)
from matrices import * 
import time 
from scipy.linalg import block_diag
from helpers import *
from warnings import warn

In [27]:
def algorithm_2(A, m, p, function, epsilon):
    '''
    A: a symmetric positive definite matrix of size n times n for some n  
    m: the chosen number of samples
    p: the chosen probabibilty for the confidence interval, the trace of function(A) will be in the bound with proba
       AT LEAST p 
    
    return: I: an unbiased estimator of the quantity tr(f(A))
            (L_p, U_p): the bounds of the confidence interval of tr(f(A)) with proba p 
    '''
    
    if np.any(A!=A.T):
        raise ValueError('The matrix isnt symmetric')
        
    if np.any(np.linalg.eigvals(A) <= 0):
        raise ValueError('The matrix isnt positive definite')
        
    if p<=0 or p>=1:
        raise ValueError('we dont have 0<p<1')
        
    if m>A.shape[0]/2: #arbitrary choice
        print('Notice that for the use of the Montecarlo method to be justified, its better to have m<=A.shape[0]/2')
        
    n=A.shape[0]
    L=np.zeros(m) 
    U=np.zeros(m) 
    L_min=10^8 #dummy value to start with for L_min
    U_max=-10^8 #dummy value to start with, for U_max
    
    
    for j in range(m):
        z = np.random.uniform(0,1,n)
        z[z<0.5]=-1
        z[z>=0.5]=1
        #apply algorithm 1 to obtain a lower bound L_j and an upper bound U_j and set 
        
        #algorithm 1 will output z.T@function(A)@z and we know from the paper that this is an unbiased estimator of 
        #tr(function(A))
        L[j], U[j]=algorithm_1(A=A, u=z, function=function, maxit=200, epsilon=epsilon)
        L_j, U_j=L[j],U[j]
        
        #computing the mean as we have an unbiased estimator
        I=np.ones(m).T@(L+U)/(2*(j+1)) #need to add +1 because of indexization in python. The first vector multiplication
                                       #sums all the non-zero elements of L+U
        L_min=np.minimum(L_min,L_j)
        U_max=np.maximum(U_max,U_j)
        eta_2=(-0.5*(j+1)*(U_max-L_min)**2)*(np.log(1-p)/2)
        
        #computing the mean as we have an unbiased estimator
        L_p_j= np.ones(m).T@L/(j+1)-np.sqrt(eta_2)/(j+1)
        U_p_j= np.ones(m).T@U/(j+1)+np.sqrt(eta_2)/(j+1)
        
    return U_p_j, L_p_j, I    

In [36]:
#the trace of the inverse is supposed to be in the interval for a given probability p but we have that our computations
#are too strong, we have that the estimation is too often correct

In [37]:
#the Poisson matrix
k=12
A= Poisson_function(k=k)
n=k**2
print("n=",n)
def f(x):
    return 1/x
tol = 1e-5
L = algorithm_2(A=A, m=50, p=0.9, function=f, epsilon=tol)

print("bounds [U,L]=", L)

I_ex=np.trace(np.linalg.inv(A))

print("exact value of trace of the inverse:", I_ex, 'found estimated value:', L[2], 'difference in two norm', np.linalg.norm(I_ex-L[2]))

# check 
print("L<I_ex:", L[1]<I_ex )
print("U>I_ex:", L[0]>I_ex )
print("L<U =", L[1]<L[0] )

n= 144
bounds [U,L]= (75.76184594849461, 54.29398826136412, 65.02791710492937)
exact value of trace of the inverse: 65.60090633777952 found estimated value: 65.02791710492937 difference in two norm 0.5729892328501478
L<I_ex: True
U>I_ex: True
L<U = True


In [49]:
#the Poisson matrix
n=120
A=Pei_function(alpha=5, n=n)

def f(x):
    return 1/x
tol = 1e-5
L = algorithm_2(A=A, m=50, p=0.9, function=f, epsilon=tol)

print("bounds [U,L]=", L)

I_ex=np.trace(np.linalg.inv(A))

print("exact value of trace of the inverse:", I_ex, 'found estimated value:', L[2], 'difference in two norm', np.linalg.norm(I_ex-L[2]))

# check 
print("L<I_ex:", L[1]<I_ex )
print("U>I_ex:", L[0]>I_ex )
print("L<U =", L[1]<L[0] )

bounds [U,L]= (26.14667462891829, 21.42554937108172, 23.786112000000003)
exact value of trace of the inverse: 23.808 found estimated value: 23.786112000000003 difference in two norm 0.02188799999999702
L<I_ex: True
U>I_ex: True
L<U = True
